In [ ]:
from pathlib import Path
import random, math, shutil, re
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import numpy as np
import pandas as pd
from datasets import Dataset, Image as HFImage

random.seed(1337)

# --- พาธหลัก (เหมือนเดิม) ---
images_root = Path("X-ray Lung Diseases Images (9 classes)")
csv_dir     = Path("prepare_data/disease_output/csv")

# --- โฟลเดอร์ปลายทางหลังบาลานซ์ (รวมรูปเดิม + รูปเสริม) ---
balanced_root = Path("lung8_balanced_1000")
balanced_root.mkdir(parents=True, exist_ok=True)

ALLOWED_CLASSES = {
    "Chest_Changes",
    "Degenerative_Infectious",
    "Higher_Density",
    "Inflammatory_Pneumonia",
    "Lower_Density",
    "Mediastinal_Changes",
    "Normal",
    "Obstructive",
}
valid_exts = {".jpg", ".jpeg", ".apng", ".bmp", ".tif", ".tiff", ".png"}  # เผื่อ .png

TARGET_PER_CLASS = 1000
MAX_ROT_DEG = 10

def load_gray_keep_size(path: Path):
    im = Image.open(path).convert("L")  # x-ray เป็น gray จะสม่ำเสมอ
    return im

def to_numpy(im: Image.Image):
    return np.array(im)

def from_numpy(arr: np.ndarray):
    return Image.fromarray(arr)

def add_gaussian_noise(im: Image.Image, std_range=(2, 8)):
    arr = to_numpy(im).astype(np.float32)
    std = random.uniform(*std_range)
    noise = np.random.normal(0, std, arr.shape).astype(np.float32)
    out = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return from_numpy(out)

def random_affine_no_flip(im: Image.Image, max_deg=MAX_ROT_DEG, max_trans=0.05, scale_range=(0.95, 1.05)):
    # หมุน ±10°, เลื่อน ≤5%, สเกล 0.95–1.05 โดย "ไม่กลับด้าน"
    w, h = im.size
    angle = random.uniform(-max_deg, max_deg)
    tx = random.uniform(-max_trans, max_trans) * w
    ty = random.uniform(-max_trans, max_trans) * h
    scale = random.uniform(*scale_range)

    # ใช้ rotate + resize + translate แบบคงขนาด
    # 1) scale
    new_w = int(w * scale)
    new_h = int(h * scale)
    im2 = im.resize((new_w, new_h), resample=Image.BICUBIC)

    # 2) pad/crop to original size
    canvas = Image.new("L", (w, h), 0)
    ox = (w - new_w) // 2
    oy = (h - new_h) // 2
    canvas.paste(im2, (ox, oy))

    # 3) rotate (fill=0 = ดำ)
    canvas = canvas.rotate(angle, resample=Image.BICUBIC, expand=False, fillcolor=0)

    # 4) translate via affine matrix
    # Affine matrix: (a, b, c, d, e, f)
    # translate = (c, f)
    canvas = canvas.transform(
        (w, h),
        Image.AFFINE,
        (1, 0, tx, 0, 1, ty),
        resample=Image.BICUBIC,
        fillcolor=0
    )
    return canvas

def safe_int(v, a, b):  # clamp
    return max(a, min(b, int(v)))

def random_crop_pad(im: Image.Image, max_ratio=0.05):
    # ครอปเล็กน้อยแล้ว pad กลับให้เท่าเดิม
    w, h = im.size
    dx = int(w * random.uniform(0, max_ratio))
    dy = int(h * random.uniform(0, max_ratio))
    left   = safe_int(dx, 0, w//2)
    right  = safe_int(w - dx, w//2, w)
    top    = safe_int(dy, 0, h//2)
    bottom = safe_int(h - dy, h//2, h)
    cropped = im.crop((left, top, right, bottom))
    return cropped.resize((w, h), Image.BICUBIC)

def enhance_random(im: Image.Image):
    # ปรับแสง/คอนทราสต์/ชาร์ปเพลส/บลอ ร์ แบบสุ่มเบาๆ
    if random.random() < 0.7:
        im = ImageEnhance.Brightness(im).enhance(random.uniform(0.9, 1.1))
    if random.random() < 0.7:
        im = ImageEnhance.Contrast(im).enhance(random.uniform(0.9, 1.1))
    if random.random() < 0.3:
        im = ImageEnhance.Sharpness(im).enhance(random.uniform(0.8, 1.2))
    if random.random() < 0.25:
        im = im.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.0, 1.2)))
    if random.random() < 0.3:
        im = add_gaussian_noise(im, std_range=(2, 6))
    if random.random() < 0.25:
        im = ImageOps.equalize(im)
    if random.random() < 0.25:
        im = ImageOps.autocontrast(im, cutoff=2)
    return im

def augment_once(im: Image.Image):
    # ลำดับแบบสุ่มเล็กน้อย (ไม่ flip)
    ops = []
    if random.random() < 0.9:
        ops.append(lambda x: random_affine_no_flip(x))
    if random.random() < 0.6:
        ops.append(lambda x: random_crop_pad(x))
    ops.append(lambda x: enhance_random(x))
    random.shuffle(ops)
    out = im
    for f in ops:
        out = f(out)
    return out

def list_images(folder: Path):
    return sorted([p for p in folder.rglob("*") if p.suffix.lower() in valid_exts and p.is_file()], key=lambda p: p.name)

def ensure_dir(d: Path):
    d.mkdir(parents=True, exist_ok=True)

def balance_class(cls_name: str):
    cls_src = images_root / cls_name
    cls_dst = balanced_root / cls_name
    ensure_dir(cls_dst)

    src_imgs = list_images(cls_src)
    n = len(src_imgs)
    if n == 0:
        print(f"[WARN] {cls_name}: ไม่มีรูป ข้าม")
        return 0

    # 1) ถ้ามากกว่า 1000 → สุ่มเลือก 1000 เพื่อบาลานซ์
    if n >= TARGET_PER_CLASS:
        pick = random.sample(src_imgs, TARGET_PER_CLASS)
        for p in pick:
            # ก็อปปี้แบบใช้ชื่อเดิม
            shutil.copy2(p, cls_dst / p.name)
        print(f"[INFO] {cls_name}: เดิม {n} → ใช้ 1000 (สุ่ม)")
        return TARGET_PER_CLASS

    # 2) ถ้าน้อยกว่า 1000 → ก็อปปี้ทั้งหมด + เติมด้วย augmentation
    for p in src_imgs:
        shutil.copy2(p, cls_dst / p.name)

    need = TARGET_PER_CLASS - n
    print(f"[INFO] {cls_name}: เดิม {n} → augment เพิ่ม {need} ให้ครบ 1000")

    # เติมแบบวนรอบแหล่งรูป
    idx = 0
    aug_id = 1
    H, W = None, None
    while need > 0:
        src_p = src_imgs[idx % n]
        im = load_gray_keep_size(src_p)
        if H is None: 
            W, H = im.size

        aug_im = augment_once(im)
        # บังคับขนาดกลับเท่าเดิม
        aug_im = aug_im.resize((W, H), Image.BICUBIC)

        # ตั้งชื่อ: <basename>_augXXXX.png
        stem = src_p.stem
        out_name = f"{stem}_aug{aug_id:04d}.png"
        out_path = cls_dst / out_name
        aug_im.save(out_path)
        aug_id += 1
        need -= 1
        idx += 1

    return TARGET_PER_CLASS

# --------- รันบาลานซ์ทุกคลาส ---------
total_after = 0
for cls in sorted(ALLOWED_CLASSES):
    total_after += balance_class(cls)
print(f"[DONE] รวมหลังบาลานซ์ = {total_after} รูป")

# =========================
#  ขั้นตอน "จับคู่รูป-ข้อความ"
# =========================

MISMATCH_POLICY = "TRUNCATE"

def pick_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

def align_texts_to_images(text_series: pd.Series, n_images: int, cls_name: str) -> pd.Series:
    s = text_series.fillna("").astype(str).reset_index(drop=True)
    n_text = len(s)

    if n_text == n_images:
        return s

    if MISMATCH_POLICY == "TRUNCATE":
        if n_text >= n_images:
            return s.iloc[:n_images].reset_index(drop=True)
        else:
            pad = pd.Series([cls_name] * (n_images - n_text))
            return pd.concat([s, pad], ignore_index=True)

    elif MISMATCH_POLICY == "CYCLE":
        if n_text == 0:
            return pd.Series([cls_name] * n_images)
        reps = (n_images + n_text - 1) // n_text
        out = pd.concat([s] * reps, ignore_index=True).iloc[:n_images]
        return out.reset_index(drop=True)

    elif MISMATCH_POLICY == "PAD_CLASS":
        if n_text >= n_images:
            return s.iloc[:n_images].reset_index(drop=True)
        else:
            pad = pd.Series([cls_name] * (n_images - n_text))
            return pd.concat([s, pad], ignore_index=True)
    else:
        raise ValueError(f"Unknown MISMATCH_POLICY: {MISMATCH_POLICY}")

def strip_aug_suffix(name: str) -> str:
    """
    รับชื่อไฟล์เช่น 'IMG001_aug0003.png' → คืน 'IMG001.png'
    """
    m = re.match(r"^(.*?)(?:_aug\d+)?(\.\w+)$", name)
    if m:
        return m.group(1) + m.group(2)
    return name

rows = []
for csv_path in sorted(csv_dir.glob("*.csv")):
    cls = csv_path.stem.strip()
    if cls not in ALLOWED_CLASSES:
        continue

    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    df["__csv_name__"] = cls

    filename_col = pick_col(df.columns, ["filename", "file", "image", "img", "path", "filepath"])
    text_col     = pick_col(df.columns, ["text", "caption", "report", "label_text", "description"])
    class_col    = pick_col(df.columns, ["class", "label", "category", "disease", "folder"])

    # รูป "หลังบาลานซ์"
    img_files = sorted(
        [p for p in (balanced_root / cls).rglob("*") if p.suffix.lower() in valid_exts and p.is_file()],
        key=lambda p: p.name
    )
    n_img = len(img_files)
    if n_img == 0:
        print(f"[WARN] โฟลเดอร์ {cls} หลังบาลานซ์ ไม่มีรูป ถูกข้าม")
        continue

    # เตรียมข้อความ
    if text_col is None:
        df["__text__"] = cls
    else:
        df["__text__"] = df[text_col].astype(str)

    # --- จับคู่ตามชื่อไฟล์ต้นฉบับ (รองรับรูปเสริม _augXXXX) ---
    if filename_col is not None:
        df = df.copy()
        df["__filename__"] = df[filename_col].astype(str).map(lambda x: Path(x).name)

        # สร้างดิกข้อความตาม "ชื่อไฟล์เดิม"
        text_by_name = (
            df.drop_duplicates(subset="__filename__")[["__filename__", "__text__"]]
              .set_index("__filename__")["__text__"]
              .to_dict()
        )

        aligned_texts = []
        for p in img_files:
            name = p.name
            base = strip_aug_suffix(name)  # ตัด _augXXXX ออกเพื่ออ้างอิงต้นฉบับ
            aligned_texts.append(text_by_name.get(base, cls))
        use_texts = pd.Series(aligned_texts, dtype=str)

    else:
        # ไม่มี filename → ใช้นโยบายยึดจำนวนรูปเป็นหลัก
        use_texts = align_texts_to_images(df["__text__"], n_img, cls)
        if len(df) != n_img:
            print(f"[INFO] {csv_path.name}: ข้อความ {len(df)} รายการ, รูป {n_img} ไฟล์ → จับคู่แบบ {MISMATCH_POLICY} เป็น {n_img}")

    out = pd.DataFrame({
        "image_path": [p.as_posix() for p in img_files],
        "text": use_texts,
        "__class__": cls
    })
    rows.append(out)

if not rows:
    raise RuntimeError("ไม่พบข้อมูลหลังประมวลผล — ตรวจโฟลเดอร์/CSV อีกครั้ง")

meta = pd.concat(rows, ignore_index=True)
meta = meta[meta["image_path"].map(lambda p: p and Path(p).exists())].drop_duplicates().reset_index(drop=True)

print("จำนวนภาพต่อคลาส (สุดท้าย):", meta["__class__"].value_counts().to_dict())

hf = Dataset.from_pandas(meta.rename(columns={"image_path": "image"}), preserve_index=False)
hf = hf.cast_column("image", HFImage())
print(hf)
hf.save_to_disk("lung8_image_text_balanced")